In [ ]:
# Fixed Kubernetes Agentic Search System

import requests
from langchain_community.document_loaders import WebBaseLoader
import xml.etree.ElementTree as ET
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from openai import OpenAI
import os

In [ ]:


def load_kubernetes_docs():
    """Load Kubernetes documentation by manually parsing the sitemap"""
    
    sitemap_url = "https://kubernetes.io/en/sitemap.xml"
    filter_prefix = "https://kubernetes.io/docs/"
    
    print("Fetching sitemap...")
    try:
        # Get the sitemap
        response = requests.get(sitemap_url, timeout=10)
        response.raise_for_status()
        
        # Parse XML using built-in ElementTree
        root = ET.fromstring(response.content)
        
        # Extract URLs from sitemap
        urls = []
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        
        for url_elem in root.findall('.//ns:url', namespace):
            loc_elem = url_elem.find('ns:loc', namespace)
            if loc_elem is not None:
                url_text = loc_elem.text
                if url_text and url_text.startswith(filter_prefix):
                    urls.append(url_text)
        
        print(f"Found {len(urls)} Kubernetes documentation URLs")
        
        # Load documents (limit for testing)
        docs = []
        max_docs = min(50, len(urls))
        
        for i, url in enumerate(urls[:max_docs]):
            try:
                print(f"Loading {i+1}/{max_docs}: {url}")
                loader = WebBaseLoader(url)
                doc_list = loader.load()
                if doc_list:
                    docs.extend(doc_list)
                    
            except Exception as e:
                print(f"  Warning: Failed to load {url}: {str(e)[:100]}")
                continue
        
        print(f"\nSuccessfully loaded {len(docs)} documents")
        return docs
        
    except Exception as e:
        print(f"Error fetching sitemap: {e}")
        return []

def load_kubernetes_docs_direct():
    """Load specific Kubernetes documentation pages directly"""
    
    key_urls = [
        "https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/",
        "https://kubernetes.io/docs/concepts/workloads/pods/",
        "https://kubernetes.io/docs/concepts/services-networking/service/",
        "https://kubernetes.io/docs/concepts/workloads/controllers/deployment/",
        "https://kubernetes.io/docs/concepts/configuration/configmap/",
        "https://kubernetes.io/docs/concepts/configuration/secret/",
        "https://kubernetes.io/docs/concepts/services-networking/ingress/",
        "https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/",
        "https://kubernetes.io/docs/concepts/workloads/controllers/daemonset/",
        "https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/",
        "https://kubernetes.io/docs/concepts/storage/volumes/",
        "https://kubernetes.io/docs/concepts/storage/persistent-volumes/",
        "https://kubernetes.io/docs/concepts/cluster-administration/namespaces/",
        "https://kubernetes.io/docs/reference/kubectl/cheatsheet/",
        "https://kubernetes.io/docs/tasks/manage-kubernetes-objects/declarative-config/",
        "https://kubernetes.io/docs/tutorials/kubernetes-basics/",
        "https://kubernetes.io/docs/concepts/security/rbac/",
        "https://kubernetes.io/docs/concepts/policy/resource-quotas/",
        "https://kubernetes.io/docs/concepts/policy/limit-ranges/",
        "https://kubernetes.io/docs/concepts/cluster-administration/networking/"
    ]
    
    docs = []
    print(f"Loading {len(key_urls)} key Kubernetes documentation pages...")
    
    for i, url in enumerate(key_urls):
        try:
            print(f"Loading {i+1}/{len(key_urls)}: {url.split('/')[-2]}")
            loader = WebBaseLoader(url)
            doc_list = loader.load()
            if doc_list:
                docs.extend(doc_list)
        except Exception as e:
            print(f"  Warning: Failed to load {url}: {str(e)[:100]}")
            continue
    
    print(f"Successfully loaded {len(docs)} key documentation pages")
    return docs

def setup_embeddings_and_vectorstore():
    """Set up embeddings and vector store"""
    
    # Set OpenAI-compatible NRP API details
    os.environ["OPENAI_API_KEY"] = "sk-XZib0Jr5mzGUgJxCDvt1Sg"
    os.environ["OPENAI_BASE_URL"] = "https://llm.nrp-nautilus.io/v1"

    # Test embedding endpoint
    print("📌 Testing 'embed-mistral' model from NRP...")
    client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_BASE_URL"]
    )

    try:
        test = client.embeddings.create(
            model="embed-mistral",
            input=["test embedding"]
        )
        print("✅ Embedding test successful. Response shape:", len(test.data[0].embedding))
    except Exception as e:
        print("❌ Embedding model 'embed-mistral' failed:", e)
        raise e

    # Initialize LangChain OpenAI-compatible embeddings
    print("🔍 Creating embeddings using NRP 'embed-mistral'...")
    embeddings = OpenAIEmbeddings(
        model="embed-mistral",
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_BASE_URL"]
    )
    
    return embeddings

def create_vectorstore(docs, embeddings):
    """Create and save vector store"""
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    print("Splitting documents into chunks...")
    texts = text_splitter.split_documents(docs)
    print(f"Created {len(texts)} text chunks")

    # Convert documents to text strings
    text_strings = [doc.page_content for doc in texts]
    
    # Build FAISS vector store
    print("📦 Building vector store...")
    vectorstore = FAISS.from_texts(texts=text_strings, embedding=embeddings)

    # Save vector store
    vectorstore.save_local("kubernetes_vectorstore")
    print("✅ Vector store saved locally.")
    
    return vectorstore

def setup_qa_chain(vectorstore):
    """Set up the QA chain with retriever"""
    
    print("\nSetting up retrieval system...")

    # Create retriever with multiple search strategies
    retriever = vectorstore.as_retriever(
        search_type="mmr",  # Maximum Marginal Relevance for diverse results
        search_kwargs={
            "k": 8,  # Retrieve top 8 most relevant chunks
            "fetch_k": 20,  # Fetch 20 candidates before MMR selection
            "lambda_mult": 0.7  # Diversity vs relevance balance
        }
    )

    # Initialize the language model (using NRP endpoint)
    llm = ChatOpenAI(
        model="gpt-4-turbo-preview",  # Change this to available model on NRP
        temperature=0.1,
        max_tokens=1000,
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_BASE_URL"]
    )

    # Create Agentic Search Template
    agentic_prompt = PromptTemplate(
        template="""You are a Kubernetes expert assistant with access to official documentation. 
Your goal is to provide accurate, helpful answers about Kubernetes concepts, troubleshooting, and best practices.

Context from Kubernetes documentation:
{context}

Human question: {question}

Instructions:
1. Analyze the question to understand what the user needs
2. Use the provided context to give accurate information
3. If the context doesn't fully answer the question, clearly state what information is missing
4. Provide practical examples when relevant
5. Suggest follow-up questions or related topics that might be helpful
6. If you detect the user might need step-by-step guidance, offer to break down complex tasks

Answer:""",
        input_variables=["context", "question"]
    )

    # Create the QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": agentic_prompt},
        return_source_documents=True
    )

    print("Agentic search system ready!")
    return qa_chain

def agentic_search(qa_chain, question, show_sources=True):
    """Perform agentic search on Kubernetes documentation"""
    
    print(f"🔍 Searching for: {question}")
    print("-" * 50)
    
    # Get answer and sources
    result = qa_chain({"query": question})
    
    print("📋 Answer:")
    print(result["result"])
    
    if show_sources and result.get("source_documents"):
        print("\n📚 Sources:")
        for i, doc in enumerate(result["source_documents"][:3], 1):
            source_url = doc.metadata.get("source", "Unknown")
            print(f"{i}. {source_url}")
            print(f"   Preview: {doc.page_content[:200]}...")
            print()
    
    return result

class KubernetesAgent:
    def __init__(self, qa_chain):
        self.qa_chain = qa_chain
        self.conversation_history = []
    
    def search_with_context(self, question):
        """Search with conversation context"""
        
        # Add conversation context to question if available
        if self.conversation_history:
            context_question = f"""
            Previous conversation context:
            {' '.join(self.conversation_history[-3:])}
            
            Current question: {question}
            """
        else:
            context_question = question
        
        result = self.qa_chain({"query": context_question})
        
        # Store in conversation history
        self.conversation_history.append(f"Q: {question}")
        self.conversation_history.append(f"A: {result['result'][:200]}...")
        
        return result
    
    def suggest_follow_ups(self, question, answer):
        """Generate follow-up questions based on the current Q&A"""
        
        follow_up_prompt = f"""
        Based on this Kubernetes Q&A:
        Q: {question}
        A: {answer[:300]}...
        
        Suggest 3 relevant follow-up questions a user might ask.
        Return only the questions, one per line.
        """
        
        try:
            follow_up_result = self.qa_chain.llm.invoke(follow_up_prompt)
            return follow_up_result.content.strip().split('\n')
        except Exception as e:
            print(f"Error generating follow-ups: {e}")
            return ["Unable to generate follow-up questions at this time."]
    
    def interactive_search(self):
        """Interactive search session"""
        print("🤖 Kubernetes Agent activated! Ask me anything about Kubernetes.")
        print("Type 'quit' to exit, 'clear' to clear history")
        
        while True:
            question = input("\n❓ Your question: ").strip()
            
            if question.lower() == 'quit':
                break
            elif question.lower() == 'clear':
                self.conversation_history = []
                print("📝 Conversation history cleared!")
                continue
            elif not question:
                continue
            
            try:
                # Get answer
                result = self.search_with_context(question)
                print(f"\n🤖 Answer: {result['result']}")
                
                # Show sources
                if result.get("source_documents"):
                    print(f"\n📚 Key sources: {len(result['source_documents'])} documents referenced")
                
                # Suggest follow-ups
                follow_ups = self.suggest_follow_ups(question, result['result'])
                print("\n💡 You might also want to ask:")
                for i, follow_up in enumerate(follow_ups[:3], 1):
                    if follow_up.strip():
                        print(f"   {i}. {follow_up.strip()}")
            except Exception as e:
                print(f"❌ Error processing your question: {e}")

def main():
    """Main function to set up and run the Kubernetes agent"""
    
    print("🚀 Initializing Kubernetes Agentic Search System...")
    
    # Step 1: Load documents
    print("📄 Loading Kubernetes documentation...")
    docs = load_kubernetes_docs()
    
    if len(docs) == 0:
        print("\nSitemap approach failed, using direct URL approach...")
        docs = load_kubernetes_docs_direct()
    
    if len(docs) == 0:
        print("❌ Failed to load any documents. Please check your internet connection.")
        return
    
    # Step 2: Set up embeddings
    print("🔧 Setting up embeddings...")
    embeddings = setup_embeddings_and_vectorstore()
    
    # Step 3: Create vector store
    print("📦 Creating vector store...")
    vectorstore = create_vectorstore(docs, embeddings)
    
    # Step 4: Set up QA chain
    print("⚙️ Setting up QA chain...")
    qa_chain = setup_qa_chain(vectorstore)
    
    # Step 5: Initialize agent
    print("🤖 Initializing Kubernetes Agent...")
    k8s_agent = KubernetesAgent(qa_chain)
    
    print("\n✅ Kubernetes Agentic Search System Ready!")
    print("\nAvailable functions:")
    print("- agentic_search(qa_chain, 'your question here')")
    print("- k8s_agent.interactive_search()  # For interactive session")
    print("- k8s_agent.search_with_context('your question')  # With conversation memory")
    
    # Test with a sample question
    test_question = "What is a Pod in Kubernetes and how does it work?"
    print(f"\n🧪 Testing with sample question: {test_question}")
    result = agentic_search(qa_chain, test_question)
    
    return qa_chain, k8s_agent

# Run the main function
if __name__ == "__main__":
    qa_chain, k8s_agent = main()